In [1]:
import matplotlib.pyplot as plt
import numpy as np
import time
from scapy.all import *
in_traffic_filename='catch_traffic_dir/in_traffic_{}.pcap'
out_traffic_filename='catch_traffic_dir/out_traffic_{}.pcap'
traffic_speed=[10,100,500,1000]
def cdf(x, plot=True, *args, **kwargs):
    x, y = sorted(x), np.arange(len(x)) / len(x)
    return plt.plot(x, y, *args, **kwargs) if plot else (x, y)

begin=time.time()
in_timestamp_list=[]
out_timestamp_list=[]
for speed in traffic_speed:
    in_pkts=rdpcap(in_traffic_filename.format(speed))
    out_pkts=rdpcap(out_traffic_filename.format(speed))
    in_timestamps=np.array([np.float64(p.time) for p in in_pkts])
    out_timestamps=np.array([np.float64(p.time) for p in out_pkts])
    in_timestamp_list.append(in_timestamps)
    out_timestamp_list.append(out_timestamps)
end=time.time()
print('use {} seconds'.format(end-begin))

in_dt_list=[]
out_dt_list=[]
delay_list=[]
scale=1e6
for i in range(0,len(traffic_speed)):
    in_,out_=in_timestamp_list[i],out_timestamp_list[i]
    in_dt=(in_[1:]-in_[:-1])*scale
    out_dt=(out_[1:]-out_[:-1])*scale
    delay=(out_-in_)*scale
    in_dt_list.append(in_dt)
    out_dt_list.append(out_dt)
    delay_list.append(delay)

plt.figure(figsize=(20,15))
for i in range(0,len(traffic_speed)):
    expected_interval=1472*8/traffic_speed[i]
    ax=plt.subplot(3,len(traffic_speed),i+1)
    cdf(in_dt_list[i],True)
    #plt.hist(in_dt_list[i],bins=100,cumulative=True,histtype='step',density=True)
    ax.title.set_text("in: speed %dMbps(us)" % traffic_speed[i])
    ax=plt.subplot(3,len(traffic_speed),i+1+len(traffic_speed))
    cdf(out_dt_list[i],True)
    #plt.plot([expected_interval, expected_interval], [0, 10000], color='r', linestyle='-', linewidth=1)
    ax.title.set_text("out: speed %dMbps(us)" % traffic_speed[i])
    ax=plt.subplot(3,len(traffic_speed),i+1+len(traffic_speed)*2)
    cdf(delay_list[i],True)
    ax.title.set_text("delay: speed %dMbps(us)" % traffic_speed[i])
#plt.savefig('iperf_packet_distribution_cdf_2.png')